## Import Packages

In [1]:
#importing some useful packages
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
import glob
%matplotlib inline

In [2]:
#Print to images next to each other:
def print2images(img1,img2):
    f, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 9))
    f.tight_layout()
    ax1.imshow(img1)
    ax1.set_title('Image1', fontsize=50)
    ax2.imshow(img2)
    ax2.set_title('Image2', fontsize=50)
    plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)

### Compute the camera calibration matrix and distortion coefficients given a set of chessboard images.

In [3]:
# Function that uses the calibration images and returns the object points and image points to calebrate an image
def calibrate():
    #reading in the path to callibration images
    images = glob.glob("camera_cal/calibration*.jpg")

    nx = 9 #number of inside corners in x
    ny = 6 #number of inside corners in y

    objectpoints=[] #3D Points in real world
    imagespoints=[] #2D Points in image plane

    #TODO: What am I doing here?
    objp=np.zeros((ny*nx,3), np.float32)
    objp[:,:2]=np.mgrid[0:nx,0:ny].T.reshape(-1,2)


    for fname in images:
        img = cv2.imread(fname) #reading in
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) #convert to gray
        ret, corners = cv2.findChessboardCorners(gray, (nx, ny), None)
        if ret: #If cornes found add images points & object points
            imagespoints.append(corners)
            objectpoints.append(objp)
    return [objectpoints,imagespoints]

# Function that takes an image performs the camera calibration, image distortion correction and 
# performs the camera calibration, image distortion correction and returns the undistorted image
def image_undistort(img, objpoints, imgpoints):
    ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, img.shape[1:], None, None)
    undist = cv2.undistort(img, mtx, dist, None, mtx)
    return undist